### install modules

In [13]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import csv
import numpy as np

### get secret

In [ ]:
import json
from pathlib import Path
from typing import Optional

BASE_DIR = Path(__file__).resolve()


def get_secret(
    key: str,
    default_value: Optional[str] = None,
    json_path: str = str(BASE_DIR / "secret.json"),
):
    with open(json_path) as f:
        secrets = json.loads(f.read())
    try:
        return secrets[key]
    except KeyError:
        if default_value:
            return default_value
        raise EnvironmentError(f"Set the {key} environment variable.")

ID = get_secret("NAVER_ID")
PW = get_secret("NAVER_PASSWORD")
        

### 웹브라우저 접속 및 로그인 

In [2]:
driver = webdriver.Chrome('/Users/aohus/Documents/workspaces/Python/Studying/chromedriver')
driver.implicitly_wait(3)

In [3]:
# 로그인 전용 화면
driver.get('https://nid.naver.com/nidlogin.login?svctype=262144&url=http://m.naver.com/aside/')

# 아이디와 비밀번호 입력
driver.find_element_by_name('id').send_keys(ID)
time.sleep(2)
driver.find_element_by_name('pw').send_keys(PW)
time.sleep(2)

# 로그인 버튼 클릭
driver.find_element_by_css_selector('#frmNIDLogin > fieldset > input').click()

### 카페 url 주소에 연결
(1) 서울지역 카테고리의 글 주소('href') 크롤링  
(2) 각 게시물의 제목, 날짜, 글쓴이, 조회수, 댓글수 크롤링

In [4]:
base_url = "https://cafe.naver.com/thebbangs?iframe_url=/ArticleList.nhn%3Fsearch.clubid=24845809%26search.menuid=25%26userDisplay=50%26search.boardtype=L%26search.specialmenutype=%26search.totalCount=501%26search.page=9"
driver.get(base_url)

In [7]:
# (폐지)구먹은빵자랑
base_url = "https://cafe.naver.com/thebbangs?iframe_url=/ArticleList.nhn%3Fsearch.clubid=24845809%26search.menuid=25%26userDisplay=50%26search.boardtype=L%26search.specialmenutype=%26search.totalCount=501%26search.page="
test_list = []
for i in range(9,141):
    base_urli = base_url + str(i)
    
    driver.get(base_urli)
    # iframe으로 프레임 전환
    driver.switch_to.frame('cafe_main')

    article_list = driver.find_elements_by_css_selector('#main-area > div:nth-child(6) > table > tbody > tr > td.td_article > div.board-list > div > a.article')
    article_urls = [ i.get_attribute('href') for i in article_list ]

    for link in article_urls:
        test_list.append(link)


6600

In [65]:
title_list = []
date_list = []
click_list = []
content_list = []
comment_list = []
name_list = []
img_list = []

for i in range(6544,len(test_list)):
    driver.get(test_list[i])
    driver.switch_to.frame('cafe_main')
    time.sleep(2.5)
    html = BeautifulSoup(driver.page_source, 'html.parser')
    
    title = html.select('#app > div > div > div.ArticleContentBox > div.article_header > div.ArticleTitle > div > h3')
    date = html.select('#app > div > div > div.ArticleContentBox > div.article_header > div.WriterInfo > div > div.article_info > span.date')
    click = html.select('#app > div > div > div.ArticleContentBox > div.article_header > div.WriterInfo > div > div.article_info > span.count')
    content = html.select('#app > div > div > div.ArticleContentBox > div.article_container > div.article_viewer > div')
    comment = html.select('#app > div > div > div.ArticleContentBox > div.article_header > div.ArticleTool > a.button_comment > strong')
    name = html.select('#app > div > div > div.ArticleContentBox > div.article_header > div.WriterInfo > div > div.profile_info > div')
    #img = html.select('')
    
    title_text = title[0].text
    date_text = date[0].text
    click_text = click[0].text
    content_text = content[0].text
    comment_text = comment[0].text
    name_text = name[0].text
    #img_text = img[0].text
    
    title_list.append(title_text)
    date_list.append(date_text)
    click_list.append(click_text)
    content_list.append(content_text)
    comment_list.append(comment_text)
    name_list.append(name_text)
    #img_list.append(img_text)
    

In [ ]:
df = pd.DataFrame(zip(title_list, date_list, click_list, content_list, comment_list, name_list))
df.columns = ['title', 'date', 'click', 'content', 'comment', 'nickname']
pd.Dataframe().to_csv('../data/posts_2015.csv')